In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./1.manufacturer.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                data = {'func': 'partsearchgetpartgroupsdd',
                        'mfr': input_.loc[a, 'Manufacturer'],
                        'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(json_data['html_replace_sections']['ddrepl[partgroup_][container]'], 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_category = [category.strip() for category in html.xpath('//select[@id="partgroup_"]/option/text()')[1:]]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Manufacturer_No.': int(input_.loc[a, 'Manufacturer_No.']),
                                        'Manufacturer': input_.loc[a, 'Manufacturer'],
                                        'Category_No.': [i+1 for i in range(len(list_category))],
                                        'Category': list_category})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Manufacturer_No.': int(input_.loc[a, 'Manufacturer_No.']),
                                     'Manufacturer': input_.loc[a, 'Manufacturer']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Manufacturer'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Manufacturer_No.', 'Category_No.'], ascending=[True, True]).reset_index(drop=True)
output_correct.to_excel('./2.category-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Manufacturer_No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./category_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：553

[ok] - 3D
[尝试次数：1] - [剩余数量：533] - [当前时间：19:04:10]

[ok] - AIRQUALITEE
[尝试次数：1] - [剩余数量：532] - [当前时间：19:04:10]

[ok] - AIRTEX
[尝试次数：1] - [剩余数量：531] - [当前时间：19:04:10]

[ok] - AISIN
[尝试次数：1] - [剩余数量：530] - [当前时间：19:04:10]

[ok] - 3M
[尝试次数：1] - [剩余数量：529] - [当前时间：19:04:10]

[ok] - ACCEL
[尝试次数：1] - [剩余数量：528] - [当前时间：19:04:10]

[ok] - ADVAN-TECH
[尝试次数：1] - [剩余数量：527] - [当前时间：19:04:10]

[ok] - AIMCO
[尝试次数：1] - [剩余数量：526] - [当前时间：19:04:10]

[ok] - AIRAID
[尝试次数：1] - [剩余数量：525] - [当前时间：19:04:10]

[ok] - AAE
[尝试次数：1] - [剩余数量：524] - [当前时间：19:04:11]

[ok] - ACI
[尝试次数：1] - [剩余数量：523] - [当前时间：19:04:11]

[ok] - AGS
[尝试次数：1] - [剩余数量：522] - [当前时间：19:04:11]

[ok] - ACDELCO
[尝试次数：1] - [剩余数量：521] - [当前时间：19:04:11]

[ok] - AEM INDUCTION
[尝试次数：1] - [剩余数量：520] - [当前时间：19:04:11]

[ok] - ACME AUTO
[尝试次数：1] - [剩余数量：519] - [当前时间：19:04:11]

[ok] - AKEBONO
[尝试次数：1] - [剩余数量：518] - [当前时间：19:04:11]

[ok] - AL'S LINER
[尝试次数：1] - [剩余数量：517] - [当前时间：19:04:11]

[ok] - ALLPARTS
[尝试次数：1] - [剩余数量：516] - [当前时间：19:04

[ok] - DUPLI-COLOR
[尝试次数：7] - [剩余数量：389] - [当前时间：19:04:32]

[ok] - ECHLIN
[尝试次数：2] - [剩余数量：388] - [当前时间：19:04:32]

[ok] - DURA-BOND
[尝试次数：4] - [剩余数量：387] - [当前时间：19:04:32]

[ok] - DETROIT IRON
[尝试次数：5] - [剩余数量：386] - [当前时间：19:04:32]

[ok] - EATON
[尝试次数：5] - [剩余数量：385] - [当前时间：19:04:32]

[ok] - EDELMANN
[尝试次数：1] - [剩余数量：384] - [当前时间：19:04:32]

[ok] - EDELBROCK
[尝试次数：2] - [剩余数量：383] - [当前时间：19:04:32]

[ok] - EIBACH
[尝试次数：2] - [剩余数量：382] - [当前时间：19:04:32]

[ok] - EBC BRAKE
[尝试次数：4] - [剩余数量：381] - [当前时间：19:04:32]

[ok] - EIS
[尝试次数：1] - [剩余数量：380] - [当前时间：19:04:33]

[ok] - EIKO
[尝试次数：2] - [剩余数量：379] - [当前时间：19:04:33]

[ok] - DEA / MARMON RIDE CONTROL
[尝试次数：18] - [剩余数量：378] - [当前时间：19:04:33]

[ok] - DEC
[尝试次数：8] - [剩余数量：377] - [当前时间：19:04:34]

[ok] - EASTERN INDUSTRIES
[尝试次数：11] - [剩余数量：376] - [当前时间：19:04:34]

[ok] - EXIDE
[尝试次数：2] - [剩余数量：375] - [当前时间：19:04:35]

[ok] - EXEDY
[尝试次数：5] - [剩余数量：374] - [当前时间：19:04:35]

[ok] - E3 SPARK PLUGS
[尝试次数：14] - [剩余数量：373] - [当前时间：19:04:35]

[ok] - DYNAM

[ok] - MAS
[尝试次数：1] - [剩余数量：249] - [当前时间：19:05:01]

[ok] - MAREMONT
[尝试次数：6] - [剩余数量：248] - [当前时间：19:05:01]

[ok] - LUK
[尝试次数：7] - [剩余数量：247] - [当前时间：19:05:01]

[ok] - LILAND GLOBAL
[尝试次数：15] - [剩余数量：246] - [当前时间：19:05:01]

[ok] - MCQUAY-NORRIS
[尝试次数：1] - [剩余数量：245] - [当前时间：19:05:01]

[ok] - MAZDA
[尝试次数：5] - [剩余数量：244] - [当前时间：19:05:02]

[ok] - MAGNUM
[尝试次数：9] - [剩余数量：243] - [当前时间：19:05:02]

[ok] - MANDO
[尝试次数：8] - [剩余数量：242] - [当前时间：19:05:02]

[ok] - MCCORD
[尝试次数：4] - [剩余数量：241] - [当前时间：19:05:02]

[ok] - MICRONAIR
[尝试次数：2] - [剩余数量：240] - [当前时间：19:05:02]

[ok] - K SOURCE
[尝试次数：35] - [剩余数量：239] - [当前时间：19:05:02]

[ok] - METRO
[尝试次数：4] - [剩余数量：238] - [当前时间：19:05:02]

[ok] - MIDAS
[尝试次数：3] - [剩余数量：237] - [当前时间：19:05:03]

[ok] - LIQUI MOLY
[尝试次数：21] - [剩余数量：236] - [当前时间：19:05:03]

[ok] - MEVOTECH
[尝试次数：5] - [剩余数量：235] - [当前时间：19:05:03]

[ok] - METRA
[尝试次数：8] - [剩余数量：234] - [当前时间：19:05:04]

[ok] - MIGHTY
[尝试次数：4] - [剩余数量：233] - [当前时间：19:05:04]

[ok] - MCGARD
[尝试次数：7] - [剩余数量：232] - [当前时间：19

[ok] - PROFORCE
[尝试次数：24] - [剩余数量：110] - [当前时间：19:05:31]

[ok] - ROADTUFF
[尝试次数：3] - [剩余数量：109] - [当前时间：19:05:32]

[ok] - PLATINUM DRIVELINE
[尝试次数：41] - [剩余数量：108] - [当前时间：19:05:32]

[ok] - RICHPORTER
[尝试次数：5] - [剩余数量：107] - [当前时间：19:05:32]

[ok] - POWER STOP
[尝试次数：1] - [剩余数量：106] - [当前时间：19:05:32]

[ok] - RDS
[尝试次数：14] - [剩余数量：105] - [当前时间：19:05:32]

[ok] - RICHMOND
[尝试次数：7] - [剩余数量：104] - [当前时间：19:05:33]

[ok] - REPLEX AUTOMOTIVE
[尝试次数：8] - [剩余数量：103] - [当前时间：19:05:33]

[ok] - REMY
[尝试次数：10] - [剩余数量：102] - [当前时间：19:05:33]

[ok] - RETECH
[尝试次数：5] - [剩余数量：101] - [当前时间：19:05:33]

[ok] - S.U.R.&R.
[尝试次数：2] - [剩余数量：100] - [当前时间：19:05:33]

[ok] - ROTOMASTER
[尝试次数：4] - [剩余数量：99] - [当前时间：19:05:33]

[ok] - ROCKAUTO
[尝试次数：1] - [剩余数量：98] - [当前时间：19:05:33]

[ok] - RADPRO
[尝试次数：19] - [剩余数量：97] - [当前时间：19:05:34]

[ok] - SAFE-LINE
[尝试次数：3] - [剩余数量：96] - [当前时间：19:05:34]

[ok] - S.A. GEAR
[尝试次数：4] - [剩余数量：95] - [当前时间：19:05:34]

[ok] - SCHRADER
[尝试次数：3] - [剩余数量：94] - [当前时间：19:05:34]

[ok] - ROL GASKET